In [1]:
import torch
import numpy as np

## Baseline Settings

MWPM. We use the open-source library PyMatching42 with the noise model used for data generation as detailed in the ‘Experimentally motivated noise model’ subsection.

BP-OSD. We use the open-source library stimbposd43. We use the exact noise model used for data generation and set the maximal belief propagation iterations to 20.

MLE. We use the algorithm developed and implemented as in ref. 14.

In [2]:
syndrome = torch.load('./cached_qec_data_small/train/syndromes_d3_c3_bs1024_D1.pt')
label = torch.load('./cached_qec_data_small/train/labels_d3_c3_bs1024_D1.pt')

In [3]:
syndrome.shape, label.shape

(torch.Size([20480, 2, 2, 8]), torch.Size([20480, 2]))

The shape is (sample, timestep, qubit, detector), the batch_size is the number of shots.

In [4]:
from mccd.dataset import load_circuit_from_file

c = load_circuit_from_file('./cached_qec_data_small/train/circuits_d3_c3_bs1024_D1.txt', 0)

In [5]:
c.n_logical_qubits

2

In [6]:
from mccd.random_clifford_circuit import *

c = TypeICircuit(n_logical_qubits=2, depth=10, circuit_index='3')

In [7]:
c.sample_circuit()

In [8]:
list(c)

[('Y', 0, [0]),
 ('X', 0, [1]),
 ('I', 1, [0]),
 ('Z', 1, [1]),
 ('Z', 2, [0]),
 ('H', 2, [1]),
 ('I', 3, [0]),
 ('I', 3, [1]),
 ('H', 4, [0]),
 ('X', 4, [1]),
 ('Z', 5, [0]),
 ('X', 5, [1]),
 ('X', 6, [0]),
 ('H', 6, [1]),
 ('Y', 7, [0]),
 ('X', 7, [1]),
 ('X', 8, [0]),
 ('Y', 8, [1]),
 ('I', 9, [0]),
 ('Z', 9, [1]),
 ('I', 9, [0]),
 ('Z', 9, [1]),
 ('X', 8, [0]),
 ('Y', 8, [1]),
 ('Y', 7, [0]),
 ('X', 7, [1]),
 ('X', 6, [0]),
 ('H', 6, [1]),
 ('Z', 5, [0]),
 ('X', 5, [1]),
 ('H', 4, [0]),
 ('X', 4, [1]),
 ('I', 3, [0]),
 ('I', 3, [1]),
 ('Z', 2, [0]),
 ('H', 2, [1]),
 ('I', 1, [0]),
 ('Z', 1, [1]),
 ('Y', 0, [0]),
 ('X', 0, [1])]

In [9]:
c.to_string_lines()

['YX', 'IZ', 'ZH', 'II', 'HX', 'ZX', 'XH', 'YX', 'XY', 'IZ']

In [ ]:
import stim


def to_stim_circuit(mccd_circuit):
    res = stim.Circuit()
    for name, moment, qubits in mccd_circuit:
        res.append(name, qubits)
    return res

from stim import Circuit


In [24]:
circuit = to_stim_circuit(c)
circuit = auto_detector(circuit, ancilla_per_round=3, rounds=10,
                        logical_targets=[0, 1], obs_index=0)

AttributeError: 'stim._stim_polyfill.Circuit' object has no attribute 'target_rec'

In [18]:
sampler = circuit.compile_detector_sampler()

In [20]:
sampler

stim.CompiledDetectorSampler(stim.Circuit('''
    Y 0
    X 1
    I 0
    Z 1 0
    H 1
    I 0 1
    H 0
    X 1
    Z 0
    X 1 0
    H 1
    Y 0
    X 1 0
    Y 1
    I 0
    Z 1
    I 0
    Z 1
    X 0
    Y 1 0
    X 1 0
    H 1
    Z 0
    X 1
    H 0
    X 1
    I 0 1
    Z 0
    H 1
    I 0
    Z 1
    Y 0
    X 1
'''))

In [19]:
shots, observables = sampler.sample(10, separate_observables=True)


In [22]:
observables

array([], shape=(10, 0), dtype=bool)

In [28]:
from mccd import CachedSyndromeDataset

dataset = CachedSyndromeDataset('./cached_qec_data_small/train/', circuit_index='3', code_distance=3, depth=3, batch_size=1024)

In [31]:
sample = dataset[0]

In [33]:
sample.keys()

dict_keys(['syndromes', 'label', 'final_round_syndromes', 'circuit'])

In [35]:
sample['syndromes'].shape, sample['label'].shape

(torch.Size([1024, 6, 2, 8]), torch.Size([1024, 2]))